In [1]:
import pickle

import cv2 as cv
import mediapipe as mp
import numpy as np

In [2]:
model_dict = pickle.load(open('./model_abc_LSM.p', 'rb'))
model = model_dict['model']

In [3]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

I0000 00:00:1707360964.221833       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-18.8.4), renderer: Intel(R) HD Graphics 6000


In [8]:
class_rgb = (('a', (255, 0, 0)),
             ('b', (235, 235, 0)),
             ('c', (0, 200, 255)),
             ('d', (170, 0, 255)),
             ('e', (255, 127, 0)),
             ('f', (191, 255, 0)),
             ('g', (0, 149, 255)),
             ('h', (255, 0, 170)),
             ('i', (255, 212, 0)),
             ('l', (106, 255, 0)),
             ('m', (0, 64, 255)),
             ('n', (237, 185, 185)),
             ('o', (185, 215, 237)),
             ('p', (200, 200, 185)),
             ('r', (220, 185, 237)),
             ('s', (165, 217, 204)),
             ('t', (143, 35, 35)),
             ('u', (35, 98, 143)),
             ('v', (143, 106, 35)),
             ('w', (107, 35, 143)),
             ('y', (79, 143, 35)))

In [27]:
cap = cv.VideoCapture(0)

while True:
    
    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,                                                    # image to draw
                hand_landmarks,                                           # model output
                mp_hands.HAND_CONNECTIONS)                                # Draw connections (default None)      connections
                #mp_drawing_styles.get_default_hand_landmarks_style()     # Colour points (default RED)          landmark_drawing_spec
                #mp_drawing_styles.get_default_hand_connections_style()   # Colour connections (default WHITE)   connection_drawing_spec
                #False                                                    # Remove points                        is_drawing_landmarks

        for hand_landmarks in results.multi_hand_landmarks:
            
            data_aux = []
            x_ = []
            y_ = []
            
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

            x1 = int(min(x_) * W) - 20
            y1 = int(min(y_) * H) - 20
    
            x2 = int(max(x_) * W) - 20
            y2 = int(max(y_) * H) - 20
            
            probabilities = model.predict_proba([np.asarray(data_aux)]) # Probabilites of each class
            prediction = np.argmax(probabilities)                       # Index of most probable class
            
            character, color = class_rgb[prediction]
            prediction_proba = str(np.max(probabilities))

            cv.rectangle(frame, (x1, y1), (x1 + 125, y1 - 45), color, -1)
            cv.rectangle(frame, (x1, y1), (x2, y2), color, 4)
            
            cv.putText(frame, character, (x1 + 10, y1 - 10), cv.FONT_HERSHEY_COMPLEX, 1.2, (255, 255, 255), 1, cv.LINE_AA)
            cv.putText(frame, prediction_proba, (x1 + 50, y1 - 10), cv.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1, cv.LINE_AA)

    cv.imshow('frame', frame)
    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()